라이브러리 임포트

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import matplotlib as mpl
import matplotlib.pyplot as plt

# save_image()를 위한 plotly 설치

!pip install plotly==4.4.1
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4

#한글 깨짐 방지
%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum
 
import matplotlib.font_manager as fm

fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

데이터 준비하기

In [ ]:
df =pd.read_csv('/content/도로교통공단_교통사고 정보.csv',encoding='euc-kr')
df.head()

In [ ]:
df.info()

발생년월일시는 날짜와 숫자로 자르기 위해서 int -> string으로 바꾼다. <br>
교통사고 시간 전처리

In [4]:
#Substr 문자열 자르기를 위해 데이터타입 변경
df = df.astype({'발생년월일시':'string'})

In [ ]:
df.info()

In [9]:
#20190101(00) -> 00 시간으로 분리
df['발생시간'] = df['발생년월일시'].str[8:]

In [ ]:
df.info()

In [10]:
#발생시간 숫자 데이터타입으로 변경
df = df.astype({'발생시간':'int64'})

In [ ]:
df.info()

In [13]:
#2019010100 -> 20190101 추출 -> 2019-01-01 (날짜형태 YYY-MM-DD 형태로 변경)
df['발생년월일시'] = pd.to_datetime(df['발생년월일시'].str[:8],
                              format='%Y-%m-%d',errors='raise')


In [ ]:
df.head()

In [ ]:
df.info()

2019년 지역별 교통사고 데이터 시각화

In [16]:
fig = px.scatter(df, x='발생년월일시', y='발생시간',
                 color='발생지시도',
                 hover_data=['발생시간'])
fig.show()

In [17]:
fig = px.scatter(df, x='발생년월일시', y='발생시간',
                 color='발생지시도',
                 size='발생시간', # 발생시간이 크다고 사망사고가 많은 것이 아니기 때문에 부적절한 size 사용
                 hover_data=['발생시간'])
fig.show()

In [23]:
fig = px.scatter(df, x='발생년월일시', y='발생시간',
                 color='발생지시도',
                 size='사망자수', 
                 hover_data=['발생시간'])
fig.show()

시간대 별 교통사고 사망자 현황

In [19]:
fig = px.bar(df,x='사망자수',y='발생시간',orientation='h') # y축으로 보는 이유는 시간이 많아서
fig.show()

시도별 교통사고 사망자 현황

In [24]:
fig = px.bar(df,x='사망자수',y='발생지시도',orientation='h',title='시도별 교통사고 사망자 현황')
# y축으로 보는 이유는 시도가 많아서
fig.show()

대구광역시의 교통사고 발생 데이터 분석

데이터 준비하기

In [ ]:
df1 = df[df.발생지시도 == '대구']
df1.info()

교통사고 발생 시간 별/구 별 사망자 현황

In [29]:
fig = px.bar(df1,x='발생시간',y='사망자수',color='발생지시군구',title='대구 교통사고 발생 시간 별/구 별 사망자 현황')
fig.show()

In [31]:
fig = px.bar(df1,x='사망자수',y='발생시간',orientation='h',color='발생지시군구',title='대구 교통사고 발생 시간 별/구 별 사망자 현황')
fig.show()

In [ ]:
fig = px.bar(df1,x='사망자수',y='발생시간',orientation='h',title='대구 교통사고 발생 시간 별/구 별 사망자 현황')
fig.show()

지도 활용하여 교통사고 현황 분석

In [34]:
import folium

map = folium.Map(location = [36.351,127.386]) #대전 시청 좌표

In [ ]:
map

지도에 필요한 정보만 담기 -> 대전의 교통사고 현황을 지도에 표출

In [ ]:
df_map =df[df.발생지시도=='대전']
df_map.info()

사망자수, 부상자수, 중상자수, 경상자수 전처리 -> 지도에 원을 그리기 위해서

In [38]:
df_map =df_map.astype({'사망자수':'float64'})
df_map =df_map.astype({'경상자수':'float64'})
df_map =df_map.astype({'중상자수':'float64'})
df_map =df_map.astype({'부상자수':'float64'})
# 버블을 그릴 때 정수형은 그릴 수 없기 때문에 잠시 float형으로 바꾼다.

In [ ]:
df_map.info()

사망자수, 경상자 수에 따른 CircleMarker 지도에 표출하기

In [42]:
for n in df_map.index:
  cnt = df_map['사망자수'][n] + df_map['경상자수'][n]
  folium.CircleMarker([df_map['위도'][n],df_map['경도'][n]],
                       radius=cnt*2,popup=df_map['사고유형'][n],color='#3186cc',fill_color='#3186cc').add_to(map)

map.save('대전교통사고현황.html')